In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ultralytics/runs

In [ ]:
# TRAINING
import logging
import os
import subprocess
import sys

import requests
import torch
from torch.cuda import is_available as cuda_is_available
from torch.utils.tensorboard import SummaryWriter


def install_requirements():
    requirements_url = "https://raw.githubusercontent.com/aai521-group6/project/main/requirements.txt"
    response = requests.get(requirements_url)
    if response.status_code == 200:
        with open("requirements.txt", "wb") as file:
            file.write(response.content)
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    else:
        raise Exception("Failed to download requirements.txt")


try:
    from ultralytics import YOLO
except ImportError:
    install_requirements()
    from ultralytics import YOLO

logging.basicConfig(level=logging.DEBUG)


class ModelManager:
    def __init__(self, model_dir="models", model_config="yolov8x.yaml", tensorboard_port=6006):
        self.setup_tensorboard()
        self.model_dir = model_dir
        self.model_config = model_config
        self.model_name = "yolov8x-coco.pt"
        self.model_path = os.path.join(self.model_dir, self.model_name)
        self.tensorboard_port = tensorboard_port
        self.ensure_model_dir_exists()
        self.download_yaml()
        self.tensorboard_logger = None

    def ensure_model_dir_exists(self):
        if not os.path.exists(self.model_dir):
            os.makedirs(self.model_dir)

    def download_yaml(self):
        yaml_file = "coco.yaml"
        if not os.path.exists(yaml_file):
            try:
                response = requests.get("https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/datasets/coco.yaml")
                response.raise_for_status()
                with open(yaml_file, "wb") as file:
                    file.write(response.content)
            except Exception as e:
                raise Exception(f"Failed to download {yaml_file}: {e}")

    def setup_tensorboard(self):
        self.tensorboard_logger = SummaryWriter("ultralytics/runs")

    def train_model(self, resume=False):
        self.ensure_model_dir_exists()
        if resume and os.path.exists(self.model_path):
            model = YOLO(self.model_path)
        else:
            model = YOLO(self.model_config)

        if cuda_is_available():
            model.cuda()

        model.train(data="coco.yaml", epochs=3, batch=1, imgsz=640, save_period=1, project=self.model_dir, name=self.model_name.replace(".pt", ""), exist_ok=True, optimizer="auto", verbose=True, plots=True, val=True, resume=resume)
        torch.save(model.state_dict(), self.model_path)

    def get_model(self):
        if not os.path.exists(self.model_path):
            self.train_model()
        return self.model_path

    def stop_tensorboard(self):
        self.tensorboard_logger.close()


ModelManager().get_model()